In [1]:
import numpy as np
import pandas as pd
import itertools

# Select sarbecovirus sequences

In [ ]:
def read_fasta(file='intermediate_fastas/aligned-sequences_noDup.fa'):
    """
    Read a FASTA file and extract sequences and names.

    Parameters:
    - file (str): Path to the FASTA file. Default is 'intermediate_fastas/aligned-sequences_noDup.fa'.

    Returns:
    - tuple: Sequences and names extracted from the FASTA file.
    """
    with open(file) as f:
        lines = f.readlines()

    names, seqs = [], []
    for line in lines:
        line_stripped = line.rstrip()
        if '>' in line_stripped:
            names.append(line_stripped[1:])
        else:
            seqs.append(line_stripped)

    return seqs, names

def create_df(seqs, names):
    """
    Create a DataFrame from sequences and names.

    Parameters:
    - seqs (list): List of sequences.
    - names (list): List of sequence names.

    Returns:
    - DataFrame: DataFrame containing sequences and names.
    """
    df = pd.DataFrame([list(seq) for seq in seqs]).T
    df.columns = names

    # Filter out columns with '-' and add a 'resid' column
    df = df[df[names[0]] != '-']
    df['resid'] = list(range(1, 1273+1))

    # Filter rows based on 'resid' range
    df = df[(df['resid'] >= 331) & (df['resid'] <= 531)]

    return df

def get_escape_fractions(classes_):
    """
    Get escape fractions from escape data based on given classes.

    Parameters:
    - classes_ (list): List of classes to filter escape data.

    Returns:
    - Series: Escape fractions calculated from the escape data.
    """
    # Read escape data from a CSV file
    escape_data = pd.read_csv('inputs/escape_data.csv')

    # Filter escape data based on given classes
    escape_data = escape_data[
        escape_data['condition_subtype'].isin(
            [f'class {class_}' for class_ in classes_]
        )
    ]

    escape_lst = []
    # Loop over antibodies
    for condition in escape_data['condition'].unique():
        df_condition = escape_data[escape_data["condition"] == condition]
        df_condition = df_condition.sort_values(by=['mut_escape'], ascending=False)

        for ind, row in df_condition.iterrows():
            condition = row['condition']
            mutation = row["wildtype"] + str(row["site"]) + row["mutation"]
            escape = row["mut_escape"]
            site = row["site"]
            escape_lst.append([condition, mutation, escape, site])

    df = pd.DataFrame(escape_lst)
    # Pivot dataframe so columns are antibodies and mutations are indices
    df_pivot = df.pivot(index=1, columns=0, values=2)
    df_pivot = df_pivot.div(df_pivot.sum(axis=0), axis=1)  # Normalize each column
    escape_fractions = df_pivot.mean(axis=1)

    return escape_fractions

In [ ]:
# Read FASTA file and create DataFrame
seqs, names = read_fasta()
df = create_df(seqs, names)

In [ ]:
# Run through and find missing mutations
escape_fractions12 = get_escape_fractions([1, 2])
escape_fractions34 = get_escape_fractions([3, 4])

missing12 = []
missing34 = []

# Iterate over columns (sequences) in the DataFrame
for col_name, seq in df.drop(columns='resid').items():
    # Iterate over residues in the sequence
    for resi_i, mutaa in enumerate(seq):
        wt = df[names[0]].iloc[resi_i]
        resid = df['resid'].iloc[resi_i]

        # Check for missing or unchanged residues
        if mutaa == '-' or wt == mutaa:
            esc12 = 0
            esc34 = escape_fractions34.max()
        else:
            mutation = wt + str(resid) + mutaa

            # Check if mutation is in a specific list
            if mutation in []:
                esc12 = 0
                esc34 = escape_fractions34.max()
            else:
                # Check if mutation is in escape_fractions12 and escape_fractions34
                if mutation in escape_fractions12:
                    esc12 = escape_fractions12[mutation]
                else:
                    missing12.append(mutation)
                    esc12 = 0  # Set a default value for missing mutations

                if mutation in escape_fractions34:
                    esc34 = escape_fractions34[mutation]
                else:
                    missing34.append(mutation)
                    esc34 = escape_fractions34.max()  # Set a default value for missing mutations


In [ ]:
# Calculate escape fractions for different classes
escape_fractions12 = get_escape_fractions([1, 2])
escape_fractions34 = get_escape_fractions([3, 4])

# Initialize dictionaries to store results
seq_dict = {}
seq_12 = {}
seq_34 = {}

# Iterate over columns (sequences) in the DataFrame
for col_name, seq in df.drop(columns='resid').items():
    esc12_tot = 0
    esc34_tot = 0
    
    # Iterate over residues in the sequence
    for resi_i, mutaa in enumerate(seq):
        wt = df[names[0]].iloc[resi_i]
        resid = df['resid'].iloc[resi_i]
        mutation = wt + str(resid) + mutaa

        # Check for missing or unchanged residues
        if mutaa == '-' or wt == mutaa or mutation in missing12:
            esc12 = 0
        else:
            esc12 = escape_fractions12[mutation]
        
        if mutaa == '-' or wt == mutaa or mutation in missing34:
            esc34 = 0
        else:
            esc34 = escape_fractions34[mutation]
            
        esc12_tot += esc12
        esc34_tot += esc34
    
    # Calculate and store results in dictionaries
    seq_dict[col_name] = (esc12_tot - esc34_tot) * 1000
    seq_12[col_name] = esc12_tot * 1000
    seq_34[col_name] = esc34_tot * 1000


In [36]:
import pickle
pickle.dump([seq_12, seq_34], open('../pickle_files/zoonotic_seqs.pkl', 'wb'))

In [ ]:
Nviruses = 10
cutoff = 40
num_sample = 10

def get_best_names(seed):
    """
    Get the best names based on escape values.

    Parameters:
    - seed (int): Random seed for reproducibility.

    Returns:
    - list: Names of the best viruses.
    """
    np.random.seed(seed)
    best_viruses_all = (
        pd.DataFrame.from_dict(seq_dict, orient='index')
        .rename(columns={0: 'escape'})
        .sort_values(by='escape', ascending=False)
    )
    sample_inds = np.random.choice(cutoff, Nviruses, replace=False)
    best_viruses = (
        best_viruses_all
        .iloc[sample_inds]
        .sort_values(by='escape', ascending=False)
    )
    indices = [names.index(best_viruses.index[i]) for i in range(len(best_viruses.index))]
    best_names = [names[i] for i in indices]
    return best_names

def get_seq_identity(df_comb):
    """
    Calculate sequence identity values between sequences in a combination.

    Parameters:
    - df_comb (DataFrame): DataFrame containing sequences for a combination.

    Returns:
    - tuple: Maximum and mean identity values.
    """
    freq_dict = {}
    for name1, seq1 in df_comb.iteritems():
        for name2, seq2 in df_comb.iteritems():
            if name1 != name2:
                count = sum(aa1 == aa2 for aa1, aa2 in zip(seq1, seq2))
                freq = count / len(seq1)
                freq_dict[f'{name1}: {name2}'] = freq
    values = np.array(list(freq_dict.values()))
    return values.max(), values.mean()

def get_best_comb(seed):
    """
    Get the best combination of sequences based on mean identity values.

    Parameters:
    - seed (int): Random seed for reproducibility.

    Returns:
    - DataFrame: Information about the best combination.
    """
    mean_identities = []
    for i, comb in enumerate(itertools.combinations(get_best_names(seed), num_sample)):
        max_, mean_ = get_seq_identity(df[list(comb)])
        mean_identities.append([comb, max_, mean_])
    
    return pd.DataFrame(mean_identities).sort_values(by=1).iloc[0]


In [ ]:
combination_similarities = []

# Iterate over a range of seeds to get combination similarities
for seed in range(10000):
    combination_similarities.append(get_best_comb(seed))

# Convert the list of combinations into a DataFrame
combination_similarities = pd.DataFrame(combination_similarities).rename(
    columns={0: 'combination names', 1: 'max_sim', 2: 'mean_sim'}
)

# Filter out combinations with identical sequences
combination_similarities = combination_similarities[
    combination_similarities['max_sim'] < 0.995
]

# Retrieve the sequence DataFrame based on the highest mean similarity
sequence_df = pd.DataFrame(combination_similarities).sort_values(by='mean_sim').iloc[1][0]
sequence_df = df[list(sequence_df) + ['resid']]

# Print sequences
for i, seq in sequence_df.drop(columns='resid').iteritems():
    print('>' + i)
    print("".join(seq[~seq.isin(['-', 'X'])].to_list()))

# Get residue numbers of final sarbecovirus sequences

In [ ]:
from Bio import SeqIO
import pandas as pd

# Read sarbecovirus sequences
sarbs = SeqIO.parse('final-sarbecovirus-sequences.fasta', 'fasta')

# Create a DataFrame from sequences and names
df = pd.DataFrame([list(seq) for seq in seqs]).T
df.columns = names

# Filter out columns with '-'
df = df[df[names[0]] != '-']

# Add a 'resid' column
df['resid'] = list(range(1, 1273 + 1))

# Iterate over sarbecovirus sequences
for sarb in sarbs:
    seq = df[sarb.description]
    seq_nodash = "".join([x for x in seq if x != '-'])

    # Find the start position
    if str(sarb.seq) in seq_nodash:
        start = seq_nodash.index(str(sarb.seq))
    else:
        print(sarb.description, '| failed to find')

    # Calculate the end position
    end = start + len(sarb.seq)

    # Print results
    print(sarb.description, '|', start + 1, end)
